# 3.3线性回归的简洁实现

In [1]:
import torch
from torch import nn
import numpy as np
torch.manual_seed(1)

print(torch.__version__)
torch.set_default_tensor_type('torch.FloatTensor')

1.3.0+cpu


## 3.3.1生成数据集

In [2]:
num_inputs = 2 
num_examples = 1000

true_w = [2, -3.4]
true_b = 4.2

features = torch.tensor(np.random.normal(0, 1, (num_examples, num_inputs)), dtype=torch.float)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float)

## 3.3.2读取数据

In [3]:
import torch.utils.data as Data

batch_size = 10

# 组合训练数据的特征和标签
dataset = Data.TensorDataset(features, labels)

# 数据集放入DataLoader，随机读取小批量
data_iter = Data.DataLoader(
    dataset=dataset,            # torch TensorDataset格式
    batch_size=batch_size,      # 小批量大小
    shuffle=True,               # 是否打乱数据
    num_workers=2,              # 多线程读数据
)

In [4]:
for X, y in data_iter:
    print(X, '\n', y)
    break

tensor([[-0.8557, -2.1054],
        [-1.7949, -0.7921],
        [ 0.1535, -1.8624],
        [ 1.0626, -0.0316],
        [ 0.0324, -1.2866],
        [ 0.2181, -0.1749],
        [ 0.3219, -0.9045],
        [ 0.8631, -0.7062],
        [ 0.8882, -0.8386],
        [-1.0002,  1.0784]]) 
 tensor([ 9.6515,  3.3196, 10.8352,  6.4168,  8.6358,  5.2047,  7.9149,  8.3289,
         8.8297, -1.4723])


## 3.3.3定义模型

In [5]:
import torch.nn as nn

In [6]:
class LinearNet(nn.Module):
    def __init__(self, n_feature):
        # 线性网络初始化的参数
        super(LinearNet, self).__init__() 
        self.linear = nn.Linear(n_feature, 1)  # `torch.nn.Linear(in_features, out_features, bias=True)`

    def forward(self, x):# 前向传播
        y = self.linear(x)
        return y
    
net = LinearNet(num_inputs) # 实例化网络
print(net)

LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


In [7]:
# 搭建网络的不同方法
# nn.Sequential是一个有序容器，网络层将按照传入nn.Sequential的顺序依次被添加到计算图中
# 写法一：不同的层作为参数，输入到nn.Sequential
net = nn.Sequential(
    nn.Linear(num_inputs, 1)
    # 此处还可以传入其他层
    )

# 写法二：调用.add_module添加网络层
net = nn.Sequential()
net.add_module('linear', nn.Linear(num_inputs, 1))
# net.add_module ......

# 写法三：将神经网络层放入一个有序字典里，再创建nn.Sequential
from collections import OrderedDict
net = nn.Sequential(OrderedDict([
          ('linear', nn.Linear(num_inputs, 1))
          # ......
        ]))

print(net)
print(net[0])

Sequential(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)
Linear(in_features=2, out_features=1, bias=True)


## 3.3.4初始化模型参数

In [8]:
from torch.nn import init

init.normal_(net[0].weight, mean=0.0, std=0.01)
init.constant_(net[0].bias, val=0.0)  # 也可使用`net[0].bias.data.fill_(0)` 直接修改bias的data

Parameter containing:
tensor([0.], requires_grad=True)

In [9]:
for param in net.parameters(): # net.parameters：模型所有的可学习参数，返回一个生成器
    print(param)
    # 不需要进行梯度的附加，init部分都已经附加上

Parameter containing:
tensor([[-0.0142, -0.0161]], requires_grad=True)
Parameter containing:
tensor([0.], requires_grad=True)


## 3.3.5定义损失函数

In [10]:
loss = nn.MSELoss() 
# `torch.nn.MSELoss(size_average=None, reduce=None, reduction='mean')`

## 3.3.6定义优化算法

In [11]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.03)
# `torch.optim.SGD(params, lr=, momentum=0, dampening=0, weight_decay=0, nesterov=False)`
print(optimizer)  

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.03
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [ ]:
# 为不同的子网络设置不同的学习率，finetune时使用
# optimizer = optim.SGD([
#     # 如果对某个参数不指定学习率，就使用最外层的默认学习率
#     {'params':net.subnet1.parameters()}, # lr = 0.03
#     {'params':net.subnet2.parameters(),'lr':0.01}
# ],lr=0.03)

In [13]:
# 调整学习率
for param_group in optimizer.param_groups:
    param_group['lr'] *=0.1

## 3.3.7训练模型

In [14]:
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        output = net(X)
        l = loss(output, y.view(-1, 1))
        optimizer.zero_grad() # 梯度清零，等价于net.zero_grad()
        l.backward()
        optimizer.step() # 迭代模型参数
    print('epoch %d, loss: %f' % (epoch, l.item()))

epoch 1, loss: 17.866367
epoch 2, loss: 5.155653
epoch 3, loss: 0.601934


In [15]:
dense = net[0]
print(true_w, dense.weight.data)
print(true_b, dense.bias.data)

[2, -3.4] tensor([[ 1.6828, -2.8981]])
4.2 tensor([3.5690])
